# Perform Backtest - Operational Congestion Management
1. Load input prepared in 'Combine input'
1. Perform backtest
1. Store backtest-output as csv in 'output'

In [8]:
import glob
import warnings
import datetime
import os
import yaml

import numpy as np
import pandas as pd

import scipy
from pathlib import Path

from tqdm import tqdm_notebook as tqdm

import cufflinks

cufflinks.go_offline()
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go

pio.renderers.default = "iframe"

from openstef.pipeline.train_create_forecast_backtest import (
    train_model_and_forecast_back_test,
)
from openstef.metrics.figure import plot_feature_importance
from openstef.data_classes.model_specifications import ModelSpecificationDataClass
from openstef.data_classes.prediction_job import PredictionJobDataClass

## Do backtest of uncurtailed load
1. Compare backtest results with operational results to ensure validity of backtest.
2. Change features of backtest to include generated wind-energy and/or solar-energy (of at least a big PV park)
3. Compare results of:
    - operational predictions
    - backtest original
    - backtest perfect wind
    - backtest perfect solar
    - backtest perfect wind+solar

In [16]:
inputs = pd.read_csv("input/prepped_inputs.csv", index_col=0, parse_dates=True)
predictors = pd.read_csv("input/predictors.csv", index_col=0, parse_dates=True).iloc[
    :, 1:
]  # first column only as reference

In [17]:
## Give variables clear names
# Uncurtailed load
unc_load = inputs["Load_corrected_for_curtailment"]
# Operational Predictions (nb. using a different correction for curtailment)
op_preds = inputs["Day_ahead_forecast"]
# True windenergy generation
wind_ref = inputs["Wind_reference"]
# PV reference profile
pv_ref = inputs["PV_reference"]

In [23]:
# Merge Target and Predictors
shared_input_data = pd.DataFrame(unc_load).merge(
    predictors, left_index=True, right_index=True
)
# Target column should be called 'load
shared_input_data = shared_input_data.rename(
    columns=dict(Load_corrected_for_curtailment="load")
)
shared_input_data.head()

,load,clouds,radiation,temp,winddeg,windspeed,windspeed_100m,pressure,humidity,rain,...,sjv_E1A,sjv_E1B,sjv_E1C,sjv_E2A,sjv_E2B,sjv_E3A,sjv_E3B,sjv_E3C,sjv_E3D,sjv_E4A
Time,,,,,,,,,,,,,,,,,,,,,
2021-02-25 12:30:00+00:00,1.59,100.0,1699921.750,8.828156,165.572105,5.257527,7.570897,102528.937500,0.846426,NaN,...,0.000028,0.000025,0.000021,0.000033,0.000031,0.000056,0.000046,0.000046,0.000030,0.0
2021-02-25 12:45:00+00:00,2.09,100.0,1683326.125,8.903656,156.779652,4.707029,6.780270,102524.113281,0.841932,NaN,...,0.000028,0.000025,0.000021,0.000033,0.000031,0.000057,0.000047,0.000047,0.000030,0.0
2021-02-25 13:00:00+00:00,2.72,100.0,1666730.500,8.979156,147.987198,4.156531,5.989644,102519.289062,0.837438,NaN,...,0.000027,0.000025,0.000022,0.000033,0.000031,0.000057,0.000046,0.000046,0.000030,0.0
2021-02-25 13:15:00+00:00,3.36,100.0,1650134.875,8.924049,143.610104,3.929555,6.120461,102532.164062,0.837671,NaN,...,0.000027,0.000025,0.000022,0.000033,0.000031,0.000057,0.000046,0.000046,0.000031,0.0
2021-02-25 13:30:00+00:00,4.83,100.0,1633539.250,8.868942,139.233009,3.702580,6.251278,102545.039062,0.837904,NaN,...,0.000027,0.000025,0.000022,0.000032,0.000031,0.000056,0.000046,0.000046,0.000030,0.0


## Do the backtests

In [24]:
## Set up backtests
# generic
# define properties of training/prediction. We call this a 'prediction_job'
shared_pj_elements = dict(
    model="xgb",
    quantiles=[0.10, 0.30, 0.50, 0.70, 0.90],
    horizon_minutes=24 * 60,
    resolution_minutes=15,
    lat=1,  # should become optional
    lon=1,  # should become optional
    forecast_type="demand",  # Note, this should become optional
)

In [27]:
result_path = Path("output") / str(datetime.datetime.utcnow()).split(".")[0].replace(
    " ", "T"
).replace(":", "-")
os.makedirs(result_path)

## Do the backtest
backtest_configs = [
    dict(
        name="backtest",
        additional_features=[],
    ),
    dict(
        name="perfect_wind",
        additional_features=[wind_ref],
    ),
    dict(
        name="perfect_solar",
        additional_features=[pv_ref],
    ),
    dict(
        name="perfect_wind_solar",
        additional_features=[wind_ref, pv_ref],
    ),
]
# Define backtest specs

# For one week per fold, use:
# n_folds = int(len(set(shared_input_data.index.date))/7.) # one week per fold
# for quicker response, use:
n_folds = 4
backtest_specs = dict(n_folds=n_folds, training_horizons=[24])
print("n_folds:", n_folds)

results = dict()
# Catch warnings for readibility
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Write settings to yaml
    with open(result_path / "configs.yaml", "w") as file:
        documents = yaml.dump({**shared_pj_elements, **backtest_specs}, file)

    for i, config in enumerate(tqdm(backtest_configs)):
        # prep input
        input_df = shared_input_data.copy(deep=True).rename(
            columns=dict(Uncurtailed_true="load")
        )
        for additional_feature in config["additional_features"]:
            input_df = input_df.merge(
                additional_feature, left_index=True, right_index=True
            )

        pj = PredictionJobDataClass(
            id=i,  # id and name should be different for each backtest
            name=config["name"],
            **shared_pj_elements,
        )
        modelspecs = ModelSpecificationDataClass(id=pj["id"])

        # perform the actual backtest
        (
            forecast,
            model,
            train_data,
            validation_data,
            test_data,
        ) = train_model_and_forecast_back_test(
            pj,
            modelspecs=modelspecs,
            input_data=input_df,
            **backtest_specs,
        )

        # Write forecast to csv
        forecast.to_csv(result_path / f'results_{config["name"]}.csv')
        train_data[0].to_csv(result_path / f'train_{config["name"]}.csv')
        validation_data[0].to_csv(result_path / f'validation_{config["name"]}.csv')

n_folds: 4


  0%|          | 0/4 [00:00<?, ?it/s]

2022-11-29 19:56.13 [info     ] Found 199 values of constant load (repeated values), converted to NaN value. cleansing_step=repeated_values frac_values=0.005182156714668889 num_values=199 pj_id=0
2022-11-29 19:56.13 [info     ] Removed 199 NaN values         num_removed_values=199
2022-11-29 19:56.33 [info     ] Postproces in preparation of storing
2022-11-29 19:56.41 [info     ] Postproces in preparation of storing
2022-11-29 19:56.49 [info     ] Postproces in preparation of storing
2022-11-29 19:57.03 [info     ] Postproces in preparation of storing
2022-11-29 19:57.09 [info     ] Found 199 values of constant load (repeated values), converted to NaN value. cleansing_step=repeated_values frac_values=0.005182156714668889 num_values=199 pj_id=1
2022-11-29 19:57.09 [info     ] Removed 199 NaN values         num_removed_values=199
2022-11-29 19:57.32 [info     ] Postproces in preparation of storing
2022-11-29 19:57.40 [info     ] Postproces in preparation of storing
2022-11-29 19:57.48 [i

KeyboardInterrupt: 

In [28]:
forecast[["forecast", "realised"]].iplot()

## Explorative plots

In [29]:
for horizon in set(forecast.horizon):
    fig = forecast.loc[
        forecast.horizon == horizon,
        [
            "quantile_P10",
            "quantile_P30",
            "quantile_P50",
            "quantile_P70",
            "quantile_P90",
            "realised",
            "forecast",
        ],
    ].iplot(asFigure=True, title=f"Horizon: {horizon}")
    fig.update_traces(
        line=dict(color="green", width=1),
        fill="tonexty",
        fillcolor="rgba(0, 255, 0, 0.1)",
        selector=lambda x: "quantile" in x.name and x.name != "quantile_P10",
    )
    fig.update_traces(
        line=dict(color="green", width=1), selector=lambda x: "quantile_P10" == x.name
    )
    fig.update_traces(
        line=dict(color="red", width=2), selector=lambda x: "realised" in x.name
    )
    fig.update_traces(
        line=dict(color="blue", width=2), selector=lambda x: "forecast" in x.name
    )
    fig.show()

## Plot relation WeatherFeatures and Observed Generation

In [31]:
## Present relations between prediction weather and observed generation
wdf = shared_input_data[["radiation", "windspeed_100m"]].merge(
    -1 * wind_ref, left_index=True, right_index=True
)
wdf = wdf.merge(-1 * pv_ref, left_index=True, right_index=True)
# drop na
wdf = wdf.dropna()
fig_layout = dict(
    width=400, height=300, template="plotly_white", margin=dict(t=0, b=0, l=0, r=0)
)
# TODO Maybe size the markers wrt the (absolute) value of the load?

# Part of timeseries for general overview
wdf.resample("1H").mean()["2021-04-06":"2021-04-10"].iplot(
    y="radiation", secondary_y="PV_reference"
)

In [105]:
rad_data = wdf.apply(lambda x: x / x.max())
rad_data = rad_data[wdf.PV_reference > 0.1]
rad_fig = rad_data.iplot(
    kind="scatter",
    x="radiation",
    y="PV_reference",
    mode="markers",
    size=1.5,
    layout=fig_layout
    | dict(
        width=320,
        height=300,
        margin=dict(t=0, b=0, l=0, r=0),
        xaxis=dict(
            title="Forecasted GHI [normalized]", tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1]
        ),
        yaxis=dict(
            title="PV Reference [normalized]", tickvals=[0, 0.2, 0.4, 0.6, 0.8, 1]
        ),
        showlegend=False,
    ),
    asFigure=True,
)
# Add linear relation
rad_fig.add_scatter(x=[0, 1], y=[0, 1], name="linear", mode="lines")

rad_fig.write_image("output/figs/rad_fig.svg")
# Calculate MAE for Radiation, where PV_ref (normalized) > 0.1
rad_threshold = 0.1
rad_mae = rad_data.diff(axis=1).iloc[:, 1].abs().mean()
print(f"Rad MAE: {rad_mae:.2f}")

Rad MAE: 0.27


In [89]:
# Repeat for windspeed
# Part of timeseries for general overview
wdf.resample("1H").mean()["2021-04-06":"2021-04-10"].iplot(
    y="windspeed_100m", secondary_y="Wind_reference"
)

In [104]:
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# fit sigmoid
def sigmoid(x, L, x0, k, b):
    y = L / (1 + np.exp(-k * (x - x0))) + b
    return y


# Normalize the windgeneration
norm_wind = wdf.copy()
norm_wind.Wind_reference /= norm_wind.Wind_reference.max()

p0 = [
    max(norm_wind.Wind_reference),
    np.median(norm_wind.windspeed_100m),
    1,
    min(norm_wind.Wind_reference),
]  # this is an mandatory initial guess
popt, pcov = curve_fit(
    sigmoid,
    norm_wind[norm_wind.Wind_reference > 0].windspeed_100m,
    norm_wind[norm_wind.Wind_reference > 0].Wind_reference,
    p0,
    method="dogbox",
)
r2 = r2_score(norm_wind.Wind_reference, sigmoid(norm_wind.windspeed_100m, *popt))

fig2 = (
    norm_wind.resample("1H")
    .mean()
    .iplot(
        x="windspeed_100m",
        y="Wind_reference",
        asFigure=True,
        mode="markers",
        size=1.5,
        layout=fig_layout
        | dict(
            xaxis=dict(title="Forecasted windspeed 100m [m/s]"),
            yaxis=dict(title="Wind Reference [normalized]"),
            width=320,
            height=300,
            margin=dict(t=0, b=0, l=0, r=0),
            showlegend=False,
        ),
    )
)

# Add fit
xs = np.linspace(0, 33, 100)
fig2.add_scatter(x=xs, y=sigmoid(xs, *popt), name="fit", mode="lines")
# fig2.add_annotation(x=0.08, xref='paper', y=0.95, yref='paper', text=f'R<sup>2</sup>: {r2:.2f}', showarrow=False)
fig2.show()

fig2.write_image("output/figs/wind_fig.svg")

## Effect of curtailments

In [22]:
# Effect of curtailments
hal[["Uncurtailed_true", "Obs"]].iplot(
    kind="hist",
    layout=dict(
        yaxis=dict(type="log", title="Count of periods [log]"),
        xaxis=dict(title="Load on Hallum [MW, 5min average]"),
        title="Effect of curtailment on load Hallum<br>Note that almost all the negative extremes are effectively reduced, while reducing the rest minimally",
        template="plotly_white",
    ),
)

In [23]:
nomargins = dict(t=0, b=0, l=0, r=0)
curtailment_threshold = -10
from plotly.subplots import make_subplots

# make a subplot with histogram of uncurtailed_moments on the right

fig = (
    hal[hal["Uncurtailed_true"] != hal["Obs"]][["Uncurtailed_true", "Obs"]]
    .rename(columns=dict(Obs="After curtailment"))
    .iplot(
        x="Uncurtailed_true",
        y="After curtailment",
        mode="markers",
        layout=dict(
            template="plotly_white",
            width=400,
            height=300,
            margin=nomargins,
            xaxis=dict(title="Uncurtailed [MW]"),
            yaxis=dict(title="Observed after curtailment [MW]"),
        ),
        size=3,
        asFigure=True,
    )
)
fig.add_scatter(
    x=[-12, 5], y=[-12, 5], mode="lines", name="Without curtailment", line=dict(width=1)
)
fig.add_hline(curtailment_threshold, line=dict(width=1))
# print('Note that the top figure only shows moments where there was curtailment in the first place. Moments where there was no curtailment are not shown')


# Add moments where there could have been curtailment but wasn't
should_have_curtailed = hal[
    (hal["Uncurtailed_true"] == hal["Obs"]) & (hal["Obs"] < (curtailment_threshold + 2))
][["Uncurtailed_true", "Obs"]].rename(columns=dict(Obs="Uncurtailed moments"))
fig.add_scatter(
    x=should_have_curtailed["Uncurtailed_true"],
    y=should_have_curtailed["Uncurtailed moments"],
    name="Uncurtailed moments",
    mode="markers",
    marker=dict(size=3),
)
fig.show()

In [24]:
curt_df = hal[["Uncurtailed_true", "Obs"]].copy()
curt_df["Curtailed moment"] = hal["Uncurtailed_true"] != hal["Obs"]
fig = px.scatter(
    curt_df,
    x="Uncurtailed_true",
    y="Obs",
    marginal_x="box",
    marginal_y="box",
    color="Curtailed moment",
)
fig = fig.update_layout(
    dict(
        template="plotly_white",
        width=500,
        height=400,
        yaxis=dict(title="Observed (after curtailment) [MW]"),
        xaxis=dict(title="Load if no curtailment took place [MW]"),
    )
)
# add linear
fig.add_scatter(
    x=[-12, 7],
    y=[-12, 7],
    mode="lines",
    showlegend=False,
    line=dict(color="black", width=1),
)

In [25]:
percentage = len(hal[hal["Uncurtailed_true"] != hal["Obs"]]) / len(hal)
print(f"Moments with curtailment = {percentage*100:.1f}%")
print(
    f"Days with curtailment = {len(set(hal[hal['Uncurtailed_true']!=hal['Obs']].index.date)) / len(set(hal.index.date))*100:.1f}%"
)

Moments with curtailment = 1.5%
Days with curtailment = 4.2%
